In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import os
import sys
from datetime import datetime
from scipy.signal import wiener

from matplotlib import colors
# for gif
import imageio
import matplotlib.cm as cm

import proplot as pplt
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False

In [ ]:
datadir = '../Diagnostics/Data/Measurements/2022-04-13/'
os.listdir(datadir)

In [ ]:
filename = '220413171330-camera_pixel_calibration'
datafile = filename+'.h5'

camname = 'cam34'
zoom_mult = 0.5

In [ ]:
# -- load file
f = h5py.File(datadir + filename + '.h5', 'r')

# -- log entries
for i in range(f['/log'].size):
    # -- only print errors/warnings
    if not(f['/log'][i,'level'] == 'INFO'.encode('utf')):
        # format timestamp
        timestr = datetime.fromtimestamp(f['/log'][0,'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        # print timestamp and message
        print('%s: %s'%(timestr,f['/log'][i,'message']))

# -- configuration data
for entry in f['/config'].keys():
    print('\n%s:'%entry)
    for name in f['/config'][entry].dtype.names:
        print('%s: %s'%(name,f['/config'][entry][name]))

In [ ]:
ds = f['/scandata']
lds = len(ds)

dt = ds.dtype
attrs = dt.names

print(attrs)

act = []
for key in attrs:
    if 'PositionSync' in key:
        act.append(key)

print(act)

## Screen edge

This method looks at the motion of slit on screen and calibrates against actuator readback.

In [ ]:
fig, ax = pplt.subplots(figsize=(3, 8))
for row in range(lds):
    ax.plot(ds[row, 'cam34_ProfileX'] * 1e-7 + row, color='black')
plt.show()

In [ ]:
pix0 = 250
pixf = 375
starti = 30
endi = lds

# -- trim up data and assign end values
dattr = ds[starti:endi, 'cam34_ProfileX'][:, pix0:pixf].copy()
#dattr.iloc[:, 0:pix0] = dattr.iloc[:, pix0].mean()
#dattr.iloc[:, pixf:] = dattr.iloc[:, pixf].mean()
slitpos = ds[starti:endi, 'screen_PositionSync']

plt.figure(figsize=[5, 10])
for row in range(np.shape(dattr)[0]):
    plt.plot(dattr[row, :] * 1e-7 + row, color='black')

In [ ]:
# -- test single wf
idy = 0
fsize = 3

# -- smooth
wf = wiener(dattr[idy,:],mysize=fsize)
# -- differentiate
dwf = wf[1:] - wf[0:-1]
# -- smooth differentiation
dwf = wiener(dwf, mysize=fsize)
# -- avg over nearest four slope points
dwf[1:] += dwf[0:-1]
dwf[0:-1] += dwf[1:]
dwf[2:] += dwf[0:-2]
dwf[0:-2] += dwf[2:]

# -- edge is point with highest slope
idx = np.argmin(dwf) - 1

plt.figure()
plt.plot(np.arange(pix0, pixf), dattr[idy, :], '.')
plt.plot(np.arange(pix0, pixf), wf, '-')
plt.plot(pix0 + idx, wf[idx], marker='s', color='black', markersize=10, label='max neg. slope')
plt.legend()

plt.figure()
plt.title('Averaged slope of waveform')
plt.plot(dwf)
plt.plot(idx + 1, dwf[idx + 1], marker='s', color='black', markersize=10, label='max neg. slope')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=[10,5])
plt.xlabel('VS pixels');
plt.ylabel('signal, Cam09:ProfileY')

iedge = np.zeros(endi-starti)
for ii in range(np.shape(dattr)[0]):
    # -- smooth
    wf = wiener(dattr[ii,:],mysize=fsize)
    # -- differentiate
    dwf = wf[1:] - wf[0:-1]
    # -- smooth differentiation
    dwf = wiener(dwf,mysize=fsize)
    # -- avg over nearest four slope points
    dwf[1:] += dwf[0:-1]
    dwf[0:-1] += dwf[1:]
    dwf[2:] += dwf[0:-2]
    dwf[0:-2] += dwf[2:]
    # -- edge is point with highest slope
    idx = np.argmin(dwf) -1
    
    # -- plot
    plt.plot(wf)
    plt.plot(idx,wf[idx],'ko')
    
    # -- save
    iedge[ii] = idx

plt.figure()
plt.plot(slitpos, iedge, '.')
plt.title('pix2mm = %.4f mm/pixel +- %.4f '%(slope, var_slope))
plt.xlabel('VS position [mm]')
plt.ylabel('Location max. neg. slope[pixels]')

pfit, COV = np.polyfit(iedge, slitpos, 1, cov=True) 
slope = pfit[0] * zoom_mult
var_slope = np.sqrt(COV[0, 0]) * zoom_mult
print('pix2mm = %.4f mm/pixel +- %.4f '%(slope, var_slope))
p = np.poly1d(pfit)
plt.plot(p(iedge), iedge);

## Calibration for 3 FODO viewscreens

In [ ]:
s = [70.31,142.31,214.31,214.31,214.31,214.31,214.31]
errs = [0.0006,0.0006,0.0027,0.0013,0.0007*3,0.0012*3,0.0009*3]
p2mm = [0.0711,0.1251,0.1930,0.1499,0.0526*3,0.0663*3,0.0597*3]

plt.errorbar(s,p2mm,yerr=np.array(errs),marker='o',markerfacecolor='none',linestyle='none',label='2/17 x0.33')


s = [70.31,142.31,214.31]
errs = [.002,.006,.008]
p2mm = [.0648,.1258,.1911]

plt.errorbar(s,p2mm,yerr=np.array(errs),marker='.',linestyle='none',label='11/11 x1.0')


s = [70.31,142.31,214.31]
p2mm = [0.0711,0.1251,0.1930]
pfit = np.polyfit(s,p2mm,1) 
slope = pfit[0]

print('pix2mm = %.4f s[inches] + %.4f '%(slope,pfit[1]))
p = np.poly1d(pfit)
plt.plot([0,215],p([0,215]));

plt.xlabel('Distance from s=0 [inches]')
plt.ylabel('pixel-to-mm conversion')

s0 = -pfit[1]/pfit[0]; 
print('crossing point at s=%.2f inches'%s0)

plt.legend()